In [ ]:
# data formatting scripts

from collections import defaultdict
import json, math, glob
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
from tqdm import tqdm

In [ ]:
def get_cruxeval(type):
    records = []
    for fname in glob.glob(f"evaluation_results/*_temp0.2_{type}.json"):
        name = fname.split('/')[1]
        model, temp, typejsonl = name.split('_')
        print(model, temp, type)

        with open(fname) as f:
            res = json.load(f)['raw_scored_generations']
            for exid in res:
                gotid = np.mean(res[exid])
                actualid = exid.replace('sample_', '')
                records.append({
                    'benchmark_id': f'CRUXEval-{type}',
                    'model': model,
                    'example_id': f"CRUXEval-{type}/{actualid}",
                    'pass1': gotid,
                    'hyperparams': 'temp0.2'
                })
    df = pd.DataFrame(records)
    display(df.describe())
    return df
        
with open('data/cruxeval_input.jsonl', 'w') as f:
    dfi = get_cruxeval('input')
    f.write(dfi.to_json(orient='records', lines=True))

with open('data/cruxeval_output.jsonl', 'w') as f:
    dfo = get_cruxeval('output')
    f.write(dfo.to_json(orient='records', lines=True))

pass1 = pd.concat([dfi, dfo])
display(pass1)


In [ ]:
# generate evalplus links

import sys
from jinja2 import Template

sys.path.append('/private/home/sida/git/CodeGen/scripts/sida/arena/data/evalplus')

from evalplus.data import get_mbpp_plus, get_human_eval_plus, write_jsonl
# has these fields dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'contract', 'base_input', 'atol', 'plus_input'])

j2_template = Template("""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8" />   <!--It is necessary to use the UTF-8 encoding with plotly graphics to get e.g. negative signs to render correctly -->
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<link
  rel="stylesheet"
  href="https://crux-eval.github.io/static/css/bulma.min.css"
>
</head>

<body>
<section class="section">                       
<h1> {{ id }} </h1>
<pre><code> {{ problem }} </code></pre>
<h2>solution</h2>
<pre><code> {{ canonical_solution }} </code></pre>
<h2>base input</h2>
<pre><code> {{ base_input }} </code></pre>
<h2>plus input</h2>
<pre><code> {{ plus_input }} </code></pre>
<br>
<h1>Generations</h1>
{% for key,value in outputs.iterrows() %}
      <p> {{ value['model'] }}. fail_base: {{ value['fail_base'] }}, fail_plus: {{ value['fail_plus'] }} </p>
      <pre><code>  {{ value['raw_genereation'] }}</code></pre>
{% endfor %}
                       
</section>

</body>
</html>
""")

def write_benchmark(benchmark_id):
    if benchmark_id == 'humaneval+':
        get_bench = get_human_eval_plus
    else:
        get_bench = get_mbpp_plus

    samples = [
        dict(example_id=task_id,
            problem=problem['prompt'],
            canonical_solution=problem['canonical_solution'],
            base_input=problem['base_input'],
            plus_input=problem['plus_input'],
        )
        for task_id, problem in get_bench().items()
    ]
    df_prob = pd.DataFrame(samples)

    records = []
    for fname in glob.glob(f"data/noise_analysis/*/*.jsonl"):
        with open(fname, 'rt') as f:
            records.extend([json.loads(l) for l in f.readlines()])

    eval_results = pd.DataFrame(records)
    display(eval_results.describe())

    benchmark_results = eval_results[eval_results['benchmark_id'] == benchmark_id]

    df = df_prob.merge(benchmark_results, on='example_id')

    ids = list(set(df['example_id']))
    for id in ids:
        current_prob = df[df['example_id'] == id]
        first = current_prob.iloc[0]
        # display(current_prob)
        generation = current_prob[['model', 'raw_genereation', 'sanitized_generation', 'fail_base', 'fail_plus']]

        with open(f'crux-eval.github.io/eval-arena/evalplus/{id}.html', 'w') as f: 
            f.write(j2_template.render({'id': id, 'problem': first['problem'],
                                        'canonical_solution': first['canonical_solution'],
                                        'base_input': first['base_input'],
                                        'plus_input': first['plus_input'],
                                        'outputs': generation}))

write_benchmark('humaneval+')
write_benchmark('mbpp+')

In [ ]:
def inspect(benchmark_id, example_id):
    # has these fields dict_keys(['task_id', 'prompt', 'entry_point', 'canonical_solution', 'test', 'contract', 'base_input', 'atol', 'plus_input'])
    if benchmark_id == 'humaneval+':
        get_bench = get_human_eval_plus
    else:
        get_bench = get_mbpp_plus

    samples = [
        dict(example_id=task_id, problem=problem['prompt'], solution=problem['canonical_solution'], test=problem['test'], plus_input=problem['plus_input'])
        for task_id, problem in get_bench().items()
    ]
    df_prob = pd.DataFrame(samples)
    for r in df_prob[df_prob['example_id'] == example_id].to_numpy():
        for v in r:
            print(v)

inspect('humaneval+', 'HumanEval/122')

In [ ]:
dfi = pd.read_csv('data/lcb_arena.csv')
display(dfi)
with open('data/lcb_arena.jsonl', 'w') as f:
    dfi = dfi[['benchmark_id', 'example_id', 'model', 'pass1']] 
    f.write(dfi.to_json(orient='records', lines=True, index=False))


In [ ]:
dfi = pd.read_json('raw-data/ds1000-full.jsonl', lines=True)
with open('data/ds1000.jsonl', 'w') as f:
    dfi = dfi[['benchmark_id', 'example_id', 'model', 'pass1']] 
    f.write(dfi.to_json(orient='records', lines=True))

In [ ]:
# process evalplus ones
def evalplus(name, isplus):
    records = []
    for fname in glob.glob(f"data/noise_analysis/{name}/*.jsonl"):
        with open(fname, 'rt') as f:
            records.extend([json.loads(l) for l in f.readlines()])
    df = pd.DataFrame(records)
    # display(df.describe())
    if isplus:
        df['pass1'] = np.where(df['fail_plus'], 0, 1) 
        df['benchmark_id'] = f'{name}+' 
    else:
        df['pass1'] = np.where(df['fail_base'], 0, 1) 
        df['benchmark_id'] = f'{name}'
    df = df[[
        'benchmark_id',
        'model',
        'example_id',
        'pass1',
    ]]
    return df

with open('data/humaneval.jsonl', 'w') as f:
    dfi = evalplus('humaneval', False)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

with open('data/humaneval+.jsonl', 'w') as f:
    dfi = evalplus('humaneval', True)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

with open('data/mbpp.jsonl', 'w') as f:
    dfi = evalplus('mbpp', False)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

with open('data/mbpp+.jsonl', 'w') as f:
    dfi = evalplus('mbpp', True)
    f.write(dfi.to_json(orient='records', lines=True, index=False))

In [ ]:

def process_llamma(bid, path, model_suf=''):
    with open(path) as f:
        lines = json.load(f)
    records = []
    for l in lines:
        if 'metadata' in l and bid == 'DS1000':
            eid = f"DS/{l['metadata']['problem_id']}"
        elif 'id' in l and bid == 'CRUXEval-input':
            eid = "CRUXEval-input/" + l['id'].split('_')[1]
        elif 'id' in l and bid == 'CRUXEval-output':
            eid = "CRUXEval-output/" + l['id'].split('_')[1]
        else:
            raise Exception('not supported') 

        records.append({
            'benchmark_id': bid,
            'example_id': eid,
            'pass1': l['metrics']['pass@1'],
            'model': f'llama3-X{model_suf}',
        })
    return records

# cruxeval_input_chat.json  cruxeval_input_cot_chat.json  cruxeval_output_chat.json  cruxeval_output_cot_chat.json  ds1000_chat.json
records = []
records += process_llamma('DS1000', 'data/llama_result/ds1000_chat.json')
records += process_llamma('CRUXEval-input', 'data/llama_result/cruxeval_input_chat.json')
records += process_llamma('CRUXEval-input', 'data/llama_result/cruxeval_input_cot_chat.json', '-cot')
records += process_llamma('CRUXEval-output', 'data/llama_result/cruxeval_output_chat.json')
records += process_llamma('CRUXEval-output', 'data/llama_result/cruxeval_output_cot_chat.json', '-cot')
dfi = pd.DataFrame(records)
with open('data/llama3.jsonl', 'wt') as f:
    f.write(dfi.to_json(orient='records', lines=True, index=False))

In [ ]:
dfi = pd.read_csv('raw-data/hf_models/human_eval.csv')
# pd.set_option('display.max_rows', 100)
# print(dfi[['model_family', 'model_name']].drop_duplicates())
# print(set(dfi['model_family']))
# filter not allowed and very bad models
dfi = dfi[~dfi['model_family'].isin(['Llama 2', 'Llama 1', 'tiiuae', 'EleutherAI'])]
maps = {'model_name': 'model', 'benchmark': 'benchmark_id', 'pass@1': 'pass1', 'example_idx': 'example_id'}
dfi = dfi[maps.keys()].rename(columns=maps)
dfi['example_id'] = dfi['example_id'].apply(lambda x: f'HumanEval/{x}')
with open('raw-data/human_eval_hf.jsonl', 'w') as f:
    dfi = dfi[['benchmark_id', 'example_id', 'model', 'pass1']] 
    f.write(dfi.to_json(orient='records', lines=True))

In [ ]:

for path in glob.glob('raw-data/hf_models/*.csv'):
    print(f'processing {path}')
    dfi = pd.read_csv(path)
    display(dfi.columns)
    bid = bprefix = set(dfi['benchmark']).pop()
    if bid == 'human_eval': # consistency of the overlap
        dfi = dfi[~dfi['model_family'].isin(['Llama 2', 'Llama 1', 'tiiuae', 'EleutherAI'])]
        bid = 'humaneval'
        dfi['benchmark'] = 'humaneval'
        bprefix = 'HumanEval'
        display(dfi)
    else:
        dfi = dfi[~dfi['model_family'].isin(['Llama 2', 'Llama 1'])]

    maps = {'model_name': 'model', 'benchmark': 'benchmark_id', 'example_idx': 'example_id'}
    if 'pass@1' in dfi.columns:
        maps['pass@1'] = 'pass1'
    elif 'acc_raw' in dfi.columns:
        maps['acc_raw'] = 'pass1'
        display(dfi[['acc_raw', 'acc_char', 'acc_token']])
        if not np.allclose(dfi['acc_raw'], dfi['acc_char'], rtol=1e-2):
            print('inconsistent')
    elif 'em' in dfi.columns:
        maps['em'] = 'pass1'

    dfi = dfi[maps.keys()].rename(columns=maps)
    display(dfi.describe())
    
    dfi['example_id'] = dfi['example_id'].apply(lambda x: f'{bprefix}/{x}')
    display(dfi)
    with open(f'raw-data/{bid}_hf.jsonl', 'w') as f:
        dfi = dfi[['benchmark_id', 'example_id', 'model', 'pass1']] 
        f.write(dfi.to_json(orient='records', lines=True))



In [ ]:
for path in glob.glob('raw-data/hf_models/human_eval.csv'):
    df = pd.read_csv(path)
    display(df.groupby('model_name').agg('mean', numeric_only=True).reset_index()[['model_name', 'pass@1']])